In [5]:
test_location = "preprocessing/data/dialogueSum/stories/test.pt"

import torch

from torch.utils.data import DataLoader

# Read the test data
test_data = torch.load(test_location)

In [6]:
len(test_data.keys())

1500

In [3]:
test_data['test_0'].keys()

dict_keys(['article', 'abstract', 'input_ids', 'attention_masks', 'cls_idx', 'abstract_vector'])

In [16]:
test_data['test_6']['article'] == test_data['test_7']['article']

True

In [9]:
test_data['test_1']['article']

"#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the

In [10]:
len(test_data['test_0']['abstract_vector'])

25

In [12]:
sum(test_data['test_0']['cls_idx'])

25

In [40]:
len(test_data['test_0']['input_ids'])

8547

In [44]:
test_data['test_0']['input_ids'][0]

101

In [11]:
from rouge_score import rouge_scorer
from models.models import Summarizer
import torch

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Summarizer(device)
model.load_state_dict(torch.load('output/model_linear.pt'))
model.to(device)
model.eval()


# Load the test data
test_data = torch.load(test_location)

sample = test_data['test_0']
print(sample['article'])
print(sample['abstract'])

# Generate the summary
summary, _ = model.predict(sample['article'])
print(summary)

# Compute the ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(sample['abstract'], summary)

print(scores)

# Now with the abstract as a paragraph
abstract = ' '.join(sample['abstract'])
scores_2 = scorer.score(abstract, summary)

print(scores_2)

FileNotFoundError: [Errno 2] No such file or directory: 'output/model_linear.pt'

In [13]:
from summarizer import Summarizer
from rouge_score import rouge_scorer
from tqdm import tqdm

model_orig = Summarizer()

summaries = {}
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'])
scores = {}
scores['rouge1'] = []
scores['rouge2'] = []
scores['rougeLsum'] = []
for key in tqdm(test_data.keys()):
    text = test_data[key]['article']
    abstract = test_data[key]['abstract']
    abstract = ' '.join(abstract)
    result = model_orig(text)
    summary = ''.join(result)
    summaries[key] = {}
    summaries[key]['real'] = abstract
    summaries[key]['predicted'] = summary
    summaries[key]['rouge_score'] = scorer.score(abstract, summaries[key]['predicted'])
    scores['rouge1'].append(summaries[key]['rouge_score']['rouge1'].fmeasure)
    scores['rouge2'].append(summaries[key]['rouge_score']['rouge2'].fmeasure)
    scores['rougeLsum'].append(summaries[key]['rouge_score']['rougeLsum'].fmeasure)

print(sum(scores['rouge1']) / len(scores['rouge1']))
print(sum(scores['rouge2']) / len(scores['rouge2']))
print(sum(scores['rougeLsum']) / len(scores['rougeLsum']))
summaries['rouge1'] = sum(scores['rouge1']) / len(scores['rouge1'])
summaries['rouge2'] = sum(scores['rouge2']) / len(scores['rouge2'])
summaries['rougeLsum'] = sum(scores['rougeLsum']) / len(scores['rougeLsum'])


  0%|          | 0/1500 [00:00<?, ?it/s]/home/jose/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  0%|          | 0/1500 [00:00<?, ?it/s]


AttributeError: module 'rouge_score.rouge_scorer' has no attribute 'score'